In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import Series, DataFrame

In [2]:
df = pd.read_csv('/Users/anilthapa/Downloads/dataset_DT.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,34.0,blue-collar,single,secondary,no,13408.0,no,no,cellular,1.0,apr,647.0,2.0,-1.0,0.0,unknown,no
1,49.0,management,divorced,tertiary,no,20727.0,no,no,cellular,1.0,apr,285.0,3.0,132.0,2.0,failure,no
2,32.0,services,single,tertiary,no,1743.0,yes,no,cellular,1.0,apr,1156.0,1.0,261.0,3.0,other,no
3,46.0,management,divorced,tertiary,no,10469.0,yes,no,cellular,1.0,apr,202.0,2.0,-1.0,0.0,unknown,no
4,26.0,management,single,tertiary,no,7628.0,no,no,cellular,1.0,apr,263.0,1.0,-1.0,0.0,unknown,no


In [9]:
df.shape

(18423, 17)

It appears that our data has 17 features, which is very high dimensional.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18423 entries, 0 to 18422
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        18423 non-null  float64
 1   job        18423 non-null  object 
 2   marital    18423 non-null  object 
 3   education  18103 non-null  object 
 4   default    18423 non-null  object 
 5   balance    18423 non-null  float64
 6   housing    18423 non-null  object 
 7   loan       18423 non-null  object 
 8   contact    18423 non-null  object 
 9   day        18117 non-null  float64
 10  month      18109 non-null  object 
 11  duration   18423 non-null  float64
 12  campaign   18423 non-null  float64
 13  pdays      18423 non-null  float64
 14  previous   18423 non-null  float64
 15  poutcome   18423 non-null  object 
 16  y          18423 non-null  object 
dtypes: float64(7), object(10)
memory usage: 2.4+ MB


In [16]:
df.marital.unique()

array(['single', 'divorced', 'married'], dtype=object)

In [17]:
df['marital'] = df['marital'].astype('category')

In [21]:
df.education.unique()

array(['secondary', 'tertiary', 'primary', 'unknown', nan], dtype=object)

In [39]:
df['job'].value_counts()

job
management       4082
blue-collar      3722
technician       2949
admin.           2012
services         1582
retired          1176
self-employed     670
entrepreneur      588
unemployed        560
student           507
housemaid         463
unknown           112
Name: count, dtype: int64

In [40]:
df['job'] = df['job'].astype('category')

In [22]:
df['education'] = df['education'].astype('category')

In [25]:
df['default'] = df['default'].astype('category')

In [27]:
df['contact'] = df['contact'].astype('category')

In [29]:
df['housing'] = df['housing'].astype('category')

In [31]:
df['loan'] = df['loan'].astype('category')

In [33]:
df['contact'] = df['contact'].astype('category')

In [35]:
df['poutcome'] = df['poutcome'].astype('category')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18423 entries, 0 to 18422
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   age        18423 non-null  float64 
 1   job        18423 non-null  category
 2   marital    18423 non-null  category
 3   education  18103 non-null  category
 4   default    18423 non-null  category
 5   balance    18423 non-null  float64 
 6   housing    18423 non-null  category
 7   loan       18423 non-null  category
 8   contact    18423 non-null  category
 9   day        18117 non-null  float64 
 10  month      18109 non-null  object  
 11  duration   18423 non-null  float64 
 12  campaign   18423 non-null  float64 
 13  pdays      18423 non-null  float64 
 14  previous   18423 non-null  float64 
 15  poutcome   18423 non-null  category
 16  y          18423 non-null  object  
dtypes: category(8), float64(7), object(2)
memory usage: 1.4+ MB


In [11]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,18423.000000,18423.000000,18117.000000,18423.000000,18423.000000,18423.000000,18423.000000
mean,40.952451,1849.616403,15.252525,411.779569,2.279433,57.421213,0.875970
std,11.458720,3742.556247,8.085925,323.241660,1.981642,112.455326,2.226526
min,18.000000,-3372.000000,1.000000,5.000000,1.000000,-1.000000,0.000000
25%,32.000000,206.000000,8.000000,192.000000,1.000000,-1.000000,0.000000
50%,38.000000,718.000000,15.000000,316.000000,2.000000,-1.000000,0.000000
75%,49.000000,2116.500000,21.000000,533.000000,3.000000,91.000000,1.000000
max,95.000000,102127.000000,31.000000,3881.000000,55.000000,871.000000,58.000000


In [4]:
df.isna().any()

age          False
job          False
marital      False
education     True
default      False
balance      False
housing      False
loan         False
contact      False
day           True
month         True
duration     False
campaign     False
pdays        False
previous     False
poutcome     False
y            False
dtype: bool

In [46]:
df.isna().sum()

age            0
job            0
marital        0
education    320
default        0
balance        0
housing        0
loan           0
contact        0
day          306
month        314
duration       0
campaign       0
pdays          0
previous       0
poutcome       0
y              0
dtype: int64

#### It appears that `education`, `day` and `month` do have missing values in the dataframe

In [8]:
df[df.isna().any(axis = 1)]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
8,33.0,admin.,single,secondary,no,285.0,yes,no,cellular,NaN,apr,427.0,1.0,329.0,1.0,other,no
57,44.0,blue-collar,married,primary,no,612.0,yes,yes,cellular,3.0,NaN,1091.0,1.0,135.0,1.0,failure,no
69,37.0,management,married,tertiary,no,489.0,yes,no,cellular,NaN,apr,246.0,2.0,133.0,6.0,failure,no
77,49.0,management,married,NaN,no,596.0,yes,no,cellular,3.0,apr,579.0,2.0,-1.0,0.0,unknown,no
88,41.0,self-employed,married,NaN,no,3123.0,yes,no,cellular,3.0,apr,1203.0,2.0,135.0,2.0,failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18324,41.0,management,married,tertiary,no,0.0,no,no,cellular,NaN,sep,294.0,4.0,63.0,11.0,other,yes
18380,32.0,management,divorced,tertiary,no,530.0,yes,no,cellular,24.0,NaN,91.0,1.0,114.0,2.0,failure,no
18389,44.0,management,married,NaN,no,1036.0,no,no,cellular,25.0,sep,491.0,1.0,-1.0,0.0,unknown,no
18392,34.0,management,married,NaN,no,812.0,no,no,cellular,27.0,sep,156.0,2.0,279.0,1.0,failure,no


In [48]:
df[df['education'].isna()]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
77,49.0,management,married,NaN,no,596.0,yes,no,cellular,3.0,apr,579.0,2.0,-1.0,0.0,unknown,no
88,41.0,self-employed,married,NaN,no,3123.0,yes,no,cellular,3.0,apr,1203.0,2.0,135.0,2.0,failure,yes
131,29.0,technician,single,NaN,no,1729.0,yes,no,cellular,6.0,apr,479.0,1.0,-1.0,0.0,unknown,no
211,42.0,blue-collar,married,NaN,no,3168.0,yes,no,cellular,8.0,apr,764.0,2.0,-1.0,0.0,unknown,no
368,40.0,self-employed,married,NaN,no,13669.0,no,no,cellular,14.0,apr,255.0,7.0,-1.0,0.0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18234,46.0,blue-collar,married,NaN,no,640.0,yes,no,cellular,11.0,sep,108.0,2.0,129.0,2.0,failure,yes
18237,28.0,blue-collar,single,NaN,no,322.0,no,no,telephone,11.0,sep,240.0,2.0,95.0,3.0,other,yes
18277,32.0,technician,single,NaN,no,1855.0,no,no,cellular,15.0,sep,126.0,2.0,134.0,1.0,other,yes
18389,44.0,management,married,NaN,no,1036.0,no,no,cellular,25.0,sep,491.0,1.0,-1.0,0.0,unknown,no


In [49]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18423 entries, 0 to 18422
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   age        18423 non-null  float64 
 1   job        18423 non-null  category
 2   marital    18423 non-null  category
 3   education  18103 non-null  category
 4   default    18423 non-null  category
 5   balance    18423 non-null  float64 
 6   housing    18423 non-null  category
 7   loan       18423 non-null  category
 8   contact    18423 non-null  category
 9   day        18117 non-null  float64 
 10  month      18109 non-null  object  
 11  duration   18423 non-null  float64 
 12  campaign   18423 non-null  float64 
 13  pdays      18423 non-null  float64 
 14  previous   18423 non-null  float64 
 15  poutcome   18423 non-null  category
 16  y          18423 non-null  object  
dtypes: category(8), float64(7), object(2)
memory usage: 1.4+ MB


In [51]:
df.select_dtypes(['category'])

,job,marital,education,default,housing,loan,contact,poutcome
0,blue-collar,single,secondary,no,no,no,cellular,unknown
1,management,divorced,tertiary,no,no,no,cellular,failure
2,services,single,tertiary,no,yes,no,cellular,other
3,management,divorced,tertiary,no,yes,no,cellular,unknown
4,management,single,tertiary,no,no,no,cellular,unknown
...,...,...,...,...,...,...,...,...
18418,technician,married,secondary,no,no,no,cellular,other
18419,management,married,unknown,no,no,no,cellular,unknown
18420,blue-collar,married,secondary,no,no,no,cellular,success
18421,technician,single,secondary,no,no,no,telephone,unknown


In [52]:
df.dtypes

age           float64
job          category
marital      category
education    category
default      category
balance       float64
housing      category
loan         category
contact      category
day           float64
month          object
duration      float64
campaign      float64
pdays         float64
previous      float64
poutcome     category
y              object
dtype: object